In [1]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Q1. Downloading the data

In [2]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [3]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
#df.info()
len(df.columns)

19

## Q2. Computing Duration

In [5]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime

In [6]:
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [7]:
df.duration.mean()

15.668995167332046

In [8]:
df.duration.std()

42.594351241920904

## Q3. Dropping outliers

In [12]:
len(df[(df.duration >= 1) & (df.duration <=60)]) / len(df) * 100

98.1220282212598

In [13]:
df_train = df[(df.duration >= 1) & (df.duration <=60)].copy()

In [15]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3009173 entries, 0 to 3066765
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

## Q4. One-hot encoding

In [16]:
cat = ['PULocationID', 'DOLocationID']

In [17]:
df_train[cat] = df_train[cat].astype(str)

In [18]:
train_dicts = df_train[cat].to_dict(orient='records')

In [21]:
len(train_dicts)

3009173

In [23]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [25]:
print(f'Feature Matrix Size: {X_train.shape}')

Feature Matrix Size: (3009173, 515)


## Q5. Training a Model

In [26]:
target = 'duration'
y_train = df_train[target].values

In [27]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

In [28]:
rmse = mean_squared_error(y_train, y_pred, squared=False)

In [29]:
print(f'Train RMSE: {rmse}')

Train RMSE: 7.6492610279057605


## Q6. Evaluating the Model

In [31]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <=60)].copy()

    cat = ['PULocationID', 'DOLocationID']
    df[cat] = df[cat].astype(str)
    
    return df

In [32]:
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [33]:
val_dicts = df_val[cat].to_dict(orient='records')

In [34]:
X_val = dv.transform(val_dicts)

In [35]:
y_val = df_val.duration.values

In [36]:
y_pred = lr.predict(X_val)

In [37]:
rmse_val = mean_squared_error(y_val, y_pred, squared=False)

In [38]:
print(f'Val RMSE: {rmse_val}')

Val RMSE: 7.81183265470218
